<a href="https://colab.research.google.com/github/MaliheDahmardeh/state-house-price-analysis-SM/blob/main/state-house-price-analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [125]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics 
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

In [126]:
!ls

sample_data  State_house_prices_ts_usa.csv


In [127]:
df = pd.read_csv('State_house_prices_ts_usa.csv')
df

,Date;RegionName;Price_1bedroom;Price_2bedroom;Price_3bedroom;Price_4bedroom;Price_5BedroomOrMore;Price_SingleFamilyResidence
0,30/04/96;Alabama;61500;48900;78200;146500;2063...
1,30/04/96;Arizona;59200;86400;96100;128400;1905...
2,30/04/96;Arkansas;53000;54500;76800;135100;186...
3,30/04/96;California;93700;123400;150900;196100...
4,30/04/96;Colorado;77800;97500;129000;176100;21...
...,...
13207,31/12/17;Virginia;200600;170300;221200;367800;...
13208,31/12/17;Washington;275700;289600;347600;46590...
13209,31/12/17;WestVirginia;;;113900;155500;181400;1...
13210,31/12/17;Wisconsin;118200;128100;169600;222600...


In [128]:
df = df['Date;RegionName;Price_1bedroom;Price_2bedroom;Price_3bedroom;Price_4bedroom;Price_5BedroomOrMore;Price_SingleFamilyResidence'].str.split(';', expand=True)
df.rename(columns = { 0:'Date', 1:'RegionName',
                                  2:'Price_1bedroom', 3:'Price_2bedroom', 4:'Price_3bedroom', 5:'Price_4bedroom', 
                                  6:'Price_5BedroomOrMore', 7:'Price_SingleFamilyResidence'}, inplace = True)
df

,Date,RegionName,Price_1bedroom,Price_2bedroom,Price_3bedroom,Price_4bedroom,Price_5BedroomOrMore,Price_SingleFamilyResidence
0,30/04/96,Alabama,61500,48900,78200,146500,206300,79000
1,30/04/96,Arizona,59200,86400,96100,128400,190500,107500
2,30/04/96,Arkansas,53000,54500,76800,135100,186000,64500
3,30/04/96,California,93700,123400,150900,196100,265300,162000
4,30/04/96,Colorado,77800,97500,129000,176100,212900,133600
...,...,...,...,...,...,...,...,...
13207,31/12/17,Virginia,200600,170300,221200,367800,531800,237300
13208,31/12/17,Washington,275700,289600,347600,465900,580300,352800
13209,31/12/17,WestVirginia,,,113900,155500,181400,103300
13210,31/12/17,Wisconsin,118200,128100,169600,222600,261200,170200


In [129]:
df.head(10)

,Date,RegionName,Price_1bedroom,Price_2bedroom,Price_3bedroom,Price_4bedroom,Price_5BedroomOrMore,Price_SingleFamilyResidence
0,30/04/96,Alabama,61500,48900,78200,146500,206300,79000
1,30/04/96,Arizona,59200,86400,96100,128400,190500,107500
2,30/04/96,Arkansas,53000,54500,76800,135100,186000,64500
3,30/04/96,California,93700,123400,150900,196100,265300,162000
4,30/04/96,Colorado,77800,97500,129000,176100,212900,133600
5,30/04/96,Connecticut,64700,97000,130400,194800,299800,141000
6,30/04/96,Delaware,90100,88200,103500,157800,176100,107400
7,30/04/96,Florida,45400,65400,89100,133600,199900,92100
8,30/04/96,Georgia,74900,64700,88000,149700,212800,92400
9,30/04/96,Hawaii,152300,186600,231800,303400,345500,262600


In [130]:
df.tail(10)

,Date,RegionName,Price_1bedroom,Price_2bedroom,Price_3bedroom,Price_4bedroom,Price_5BedroomOrMore,Price_SingleFamilyResidence
13202,31/12/17,SouthDakota,115600,157300,180500,213700,277400,178700
13203,31/12/17,Tennessee,140800,115900,163900,270900,396100,145000
13204,31/12/17,Texas,,,166200,264900,,175000
13205,31/12/17,Utah,,197600,248200,286300,355300,270600
13206,31/12/17,Vermont,174700,213800,246400,310800,367900,224700
13207,31/12/17,Virginia,200600,170300,221200,367800,531800,237300
13208,31/12/17,Washington,275700,289600,347600,465900,580300,352800
13209,31/12/17,WestVirginia,,,113900,155500,181400,103300
13210,31/12/17,Wisconsin,118200,128100,169600,222600,261200,170200
13211,31/12/17,Wyoming,,160800,211300,236200,285200,200500


In [131]:
df2=df.mask(df == '')
df2

,Date,RegionName,Price_1bedroom,Price_2bedroom,Price_3bedroom,Price_4bedroom,Price_5BedroomOrMore,Price_SingleFamilyResidence
0,30/04/96,Alabama,61500,48900,78200,146500,206300,79000
1,30/04/96,Arizona,59200,86400,96100,128400,190500,107500
2,30/04/96,Arkansas,53000,54500,76800,135100,186000,64500
3,30/04/96,California,93700,123400,150900,196100,265300,162000
4,30/04/96,Colorado,77800,97500,129000,176100,212900,133600
...,...,...,...,...,...,...,...,...
13207,31/12/17,Virginia,200600,170300,221200,367800,531800,237300
13208,31/12/17,Washington,275700,289600,347600,465900,580300,352800
13209,31/12/17,WestVirginia,NaN,NaN,113900,155500,181400,103300
13210,31/12/17,Wisconsin,118200,128100,169600,222600,261200,170200


In [132]:
df.dtypes

Date                           object
RegionName                     object
Price_1bedroom                 object
Price_2bedroom                 object
Price_3bedroom                 object
Price_4bedroom                 object
Price_5BedroomOrMore           object
Price_SingleFamilyResidence    object
dtype: object

In [137]:
df['Price_1bedroom'] = df['Price_1bedroom'].apply(pd.to_numeric, errors='coerce')
df['Price_2bedroom'] = df['Price_2bedroom'].apply(pd.to_numeric, errors='coerce')
df['Price_3bedroom'] = df['Price_3bedroom'].apply(pd.to_numeric, errors='coerce')
df['Price_4bedroom'] = df['Price_4bedroom'].apply(pd.to_numeric, errors='coerce')
df['Price_5BedroomOrMore'] = df['Price_5BedroomOrMore'].apply(pd.to_numeric, errors='coerce')
df['Price_SingleFamilyResidence'] = df['Price_SingleFamilyResidence'].apply(pd.to_numeric, errors='coerce')

In [138]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13212 entries, 0 to 13211
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Date                         13212 non-null  object 
 1   RegionName                   13212 non-null  object 
 2   Price_1bedroom               10605 non-null  float64
 3   Price_2bedroom               11745 non-null  float64
 4   Price_3bedroom               12787 non-null  float64
 5   Price_4bedroom               12359 non-null  float64
 6   Price_5BedroomOrMore         11814 non-null  float64
 7   Price_SingleFamilyResidence  12438 non-null  float64
dtypes: float64(6), object(2)
memory usage: 825.9+ KB


In [139]:
df.describe()

,Price_1bedroom,Price_2bedroom,Price_3bedroom,Price_4bedroom,Price_5BedroomOrMore,Price_SingleFamilyResidence
count,10605.000000,11745.000000,12787.000000,12359.000000,1.181400e+04,12438.000000
mean,117060.424328,135168.803746,167062.798154,243829.670685,3.237335e+05,174154.381733
std,61056.036728,72267.645405,80212.505902,106216.755129,1.702917e+05,92243.367198
min,30900.000000,32800.000000,49600.000000,64700.000000,6.860000e+04,37900.000000
25%,74600.000000,86700.000000,116400.000000,174900.000000,2.179000e+05,115000.000000
50%,100400.000000,115400.000000,141200.000000,218000.000000,2.880000e+05,147300.000000
75%,142300.000000,166800.000000,204400.000000,281000.000000,3.653000e+05,211775.000000
max,390200.000000,542400.000000,639700.000000,850400.000000,1.497000e+06,737500.000000


In [140]:
df.duplicated().sum()   

0

In [142]:
df.isnull().sum()

Date                              0
RegionName                        0
Price_1bedroom                 2607
Price_2bedroom                 1467
Price_3bedroom                  425
Price_4bedroom                  853
Price_5BedroomOrMore           1398
Price_SingleFamilyResidence     774
dtype: int64